In [1]:
import os, sys, random

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

import wandb

wandb.login()

wandb: Currently logged in as: philliphu (use `wandb login --relogin` to force relogin)


True

In [2]:
rootDir = "/Users/phillip/Documents/PhD/PhD3/Data/PhysioNet/dataset"
df_list = []

i = 0
files = os.listdir(rootDir)
for file in files:
    df_list.append(pd.read_csv(os.path.join(rootDir, file), sep="|", header=0))

In [3]:
# 计算所有数据的平均值填充
# cell001_1
s = np.zeros(40).tolist()
c = np.zeros(40).tolist()

for e in df_list:
    s = s + e.sum()[:-1]
    c = c + e.count()[:-1]

a = s.div(c)

# 当不能用线性填充时,填充最近的,再不行用平均值填充
# cell001_2
threshold = 0  # the healthy-sepsis ratio threshold
data = []
i = 0
for e in df_list:
    # t = e.iloc[:, -1][e.iloc[:, -1] == 1].sum()
    # l = len(e)
    # if t / l > threshold:
    e.interpolate(inplace=True, limit_direction="both")
    e.interpolate(inplace=True, method="nearest")
    na_cols = e.columns[e.isna().any()]
    for col in na_cols:
        e.loc[e[col].isnull(), col] = a[col]
    data.append(e)
    # i+=1
    # if i%1000==0:
    #     print(i)

In [4]:
# 生成bag
bags = []

for i in range(len(data)):
    # l = random.randint(3,5)
    l = 4
    cur = 0
    while cur + l < len(data[i]):
        bags.append(data[i].iloc[cur : cur + l, :])
        # y_bags.append((data[i].iloc[cur:cur+l,-1] > 0).any())
        cur = cur + l

In [5]:
X_s = []
X_n = []

for i in range(len(bags)):
    if (bags[i].iloc[:, -1] > 0).any():
        X_s.append(bags[i])
    else:
        X_n.append(bags[i])
    # if i%10000==0:
    #     print(i)

X_n = random.sample(X_n, len(X_s))
X_n.extend(X_s)
data = X_n

In [6]:
# Normalise
train, test = train_test_split(data)

merged_train_X = pd.concat(train).iloc[:, :-1]
merged_test_X = pd.concat(test).iloc[:, :-1]

scaler = preprocessing.StandardScaler()
scaled_train_X = scaler.fit_transform(merged_train_X)

start_index = 0

X_train = pd.concat(train).iloc[:, :-1]
y_train = pd.concat(train).iloc[:, -1]
X_test = pd.concat(test).iloc[:, :-1]
y_test = pd.concat(test).iloc[:, -1]

In [30]:
wandb.init(
    name="RF - n_estimator:10 - max_depth:5",
    project="Thesis-MIL",
    tags=["RF"],
    id="0.3",
)

In [32]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10, max_depth=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_probas = clf.predict_proba(X_test)

In [37]:
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    classification_report,
)

In [10]:
wandb.sklearn.plot_learning_curve(clf, X_train, y_train)

In [13]:
wandb.sklearn.plot_roc(y_test, y_probas)

In [18]:
wandb.sklearn.plot_confusion_matrix(y_test, y_pred)

In [19]:
wandb.sklearn.plot_summary_metrics(clf, X_train, y_train, X_test, y_test)

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. ... 1. 1. 1.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [33]:
wandb.sklearn.plot_classifier(
    clf,
    X_train,
    X_test,
    y_train,
    y_test,
    y_pred,
    y_probas,
    labels=["normal", "sepsis"],
    model_name="RF",
    feature_names=None,
)

wandb: 
wandb: Plotting RF.
wandb: Logged feature importances.
wandb: Logged learning curve.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision recall curve.


In [35]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.7071240105540897

In [36]:
roc_auc_score(y_true=y_test, y_score=y_probas[:, 1])

0.7744693082922399

In [39]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.71      0.84      0.77      7090
           1       0.70      0.52      0.60      5038

    accuracy                           0.71     12128
   macro avg       0.70      0.68      0.68     12128
weighted avg       0.71      0.71      0.70     12128



In [34]:
wandb.finish()

_runtime,357
_timestamp,1621425752
_step,10


_runtime,▁▁▁▇▇██████
_timestamp,▁▁▁▇▇██████
_step,▁▂▂▃▄▅▅▆▇▇█
